# FP-growth
FP-Growth(频繁模式增长)算法是韩家炜老师在2000年提出的关联分析算法，它采取如下分治策略：将提供频繁项集的数据库压缩到一棵频繁模式树（FP-Tree），但仍保留项集关联信息；该算法和Apriori算法最大的不同有两点：第一，不产生候选集，第二，只需要两次遍历数据库，大大提高了效率。

## 演示—构造FP-树
***（1）事务数据库建立 ***  
原始事务数据库如下：
<table border="1" width="70%" style="margin-left:0px">
<thead>
    <tr>
      <th>Tid</th>
      <th>items</th>
    </tr>
</thead>
<tr> <td> 1 </td><td> I1,I2,I5 </td> </tr>
<tr> <td> 2 </td><td> I2,I4 </td> </tr>
<tr> <td> 3 </td><td> I2,I3 </td> </tr>
<tr> <td> 4 </td><td> I1,I2,I4 </td> </tr>
<tr> <td> 5 </td><td> I1,I3 </td> </tr>
<tr> <td> 6 </td><td> I2,I3 </td> </tr>
<tr> <td> 7 </td><td>I1,I3 </td> </tr>
<tr> <td> 8 </td><td> I1,I2,I3,I5 </td> </tr>
<tr> <td> 9 </td><td> I1,I2,I3 </td> </tr>
</table>

扫描事务数据库得到频繁1-项目集F。  
I1:6 I2:7 I3:6  I4:2  I5:2    
定义minsup=20%，即最小支持度为2，重新排列F。
I2:7  I1:6  I3:6  I4:2  I5:2  
重新调整事务数据库。
<table border="1" width="70%" style="margin-left:0px">
<thead>
    <tr>
      <th>Tid</th>
      <th>items</th>
    </tr>
</thead>
<tr> <td> 1 </td><td> I2, I1,I5 </td> </tr>
<tr> <td> 2 </td><td> I2,I4 </td> </tr>
<tr> <td> 3 </td><td> I2,I3 </td> </tr>
<tr> <td> 4 </td><td> I2, I1,I4 </td> </tr>
<tr> <td> 5 </td><td> I1,I3 </td> </tr>
<tr> <td> 6 </td><td> I2,I3 </td> </tr>
<tr> <td> 7 </td><td> I1,I3 </td> </tr>
<tr> <td> 8 </td><td> I2, I1,I3,I5 </td> </tr>
<tr> <td> 9 </td><td> I2, I1,I3 </td> </tr>
</table>

*** （2）创建根结点和频繁项目表 ***
<img src="./images/fp/1.png" width = "500px" style="margin-left:0px"/>
***（3）加入第一个事务(I2,I1,I5) ***
<img src="./images/fp/2.png" width = "500px" style="margin-left:0px"/>
*** （4）加入第二个事务(I2,I4) ***
<img src="./images/fp/3.png" width = "500px" style="margin-left:0px"/>
*** （5）加入第三个事务(I2,I3) ***
<img src="./images/fp/4.png" width = "500px" style="margin-left:0px"/>
以此类推加入第5、6、7、8、9个事务。
*** （6）加入第九个事务(I2,I1,I3) ***
<img src="./images/fp/5.png" width = "500px" style="margin-left:0px"/>

## 演示—FP-树挖掘
FP-树建好后，就可以进行频繁项集的挖掘，挖掘算法称为FpGrowth（Frequent Pattern Growth）算法，挖掘从表头header的最后一个项开始，以此类推。本文以I5、I3为例进行挖掘。  
*** （1）挖掘I5： ***  
对于I5，得到条件模式基：<(I2,I1:1)>、<I2,I1,I3:1>  
构造条件FP-tree：
<img src="./images/fp/6.png" width = "500px" style="margin-left:0px"/>  
得到I5频繁项集：{{I2,I5:2},{I1,I5:2},{I2,I1,I5:2}}  
I4、I1的挖掘与I5类似，条件FP-树都是单路径。  
***（2）挖掘I3：***  
I5的情况是比较简单的，因为I5对应的条件FP-树是单路径的，I3稍微复杂一点。I3的条件模式基是(I2 I1:2), (I2:2), (I1:2)，生成的条件FP-树如下图：
<img src="./images/fp/7.png" width = "500px" style="margin-left:0px"/>  
I3的条件FP-树仍然是一个多路径树，首先把模式后缀I3和条件FP-树中的项头表中的每一项取并集，得到一组模式{I2 I3:4, I1 I3:4}，但是这一组模式不是后缀为I3的所有模式。还需要递归调用FP-growth，模式后缀为{I1，I3}，{I1，I3}的条件模式基为{I2：2}，其生成的条件FP-树如下图所示。  
<img src="./images/fp/8.png" width = "500px" style="margin-left:0px"/>  
在FP_growth中把I2和模式后缀{I1，I3}取并得到模式{I1 I2 I3：2}。

理论上还应该计算一下模式后缀为{I2，I3}的模式集，但是{I2，I3}的条件模式基为空，递归调用结束。最终模式后缀I3的支持度>2的所有模式为：{ I2 I3:4, I1 I3:4, I1 I2 I3:2}。 

## Spark MLlib 中的 FP-growth 算法参数：
* minSupport: 最小支持度
* numPartitions: 分布式运行情况下分区数  

## 例子（Examples）
FPGrowth 实现了 FP-growth 算法。该算法采用的是 RDD 事务每个 RDD 都是 Array 类型，运行该事务就返回 FPGrowthModel 用该模型存储频繁项集和它们的频率。下面的示例演示如何从事务中挖掘频繁项集和关联规则。

In [3]:
val PATH = "file:///Users/lzz/work/SparkML/"
import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.fpm.FPGrowth

val data = sc.textFile(PATH+"data/mllib/sample_fpgrowth.txt")

val transactions: RDD[Array[String]] = data.map(s => s.trim.split(' '))

val fpg = new FPGrowth().setMinSupport(0.2).setNumPartitions(10)
val model = fpg.run(transactions)
  
model.freqItemsets.collect().foreach { itemset =>
  println(itemset.items.mkString("[", ",", "]") + ", " + itemset.freq)
}

val minConfidence = 0.8
model.generateAssociationRules(minConfidence).collect().foreach { rule =>
  println(
    rule.antecedent.mkString("[", ",", "]")
      + " => " + rule.consequent .mkString("[", ",", "]")
      + ", " + rule.confidence)
}

[z], 5
[x], 4
[x,z], 3
[y], 3
[y,x], 3
[y,x,z], 3
[y,z], 3
[r], 3
[r,x], 2
[r,z], 2
[s], 3
[s,y], 2
[s,y,x], 2
[s,y,x,z], 2
[s,y,z], 2
[s,x], 3
[s,x,z], 2
[s,z], 2
[t], 3
[t,y], 3
[t,y,x], 3
[t,y,x,z], 3
[t,y,z], 3
[t,s], 2
[t,s,y], 2
[t,s,y,x], 2
[t,s,y,x,z], 2
[t,s,y,z], 2
[t,s,x], 2
[t,s,x,z], 2
[t,s,z], 2
[t,x], 3
[t,x,z], 3
[t,z], 3
[p], 2
[p,r], 2
[p,r,z], 2
[p,z], 2
[q], 2
[q,y], 2
[q,y,x], 2
[q,y,x,z], 2
[q,y,z], 2
[q,t], 2
[q,t,y], 2
[q,t,y,x], 2
[q,t,y,x,z], 2
[q,t,y,z], 2
[q,t,x], 2
[q,t,x,z], 2
[q,t,z], 2
[q,x], 2
[q,x,z], 2
[q,z], 2
[t,s,y] => [x], 1.0
[t,s,y] => [z], 1.0
[y,x,z] => [t], 1.0
[y] => [x], 1.0
[y] => [z], 1.0
[y] => [t], 1.0
[p] => [r], 1.0
[p] => [z], 1.0
[q,t,z] => [y], 1.0
[q,t,z] => [x], 1.0
[q,y] => [x], 1.0
[q,y] => [z], 1.0
[q,y] => [t], 1.0
[t,s,x] => [y], 1.0
[t,s,x] => [z], 1.0
[q,t,y,z] => [x], 1.0
[q,t,x,z] => [y], 1.0
[q,x] => [y], 1.0
[q,x] => [t], 1.0
[q,x] => [z], 1.0
[t,x,z] => [y], 1.0
[x,z] => [y], 1.0
[x,z] => [t], 1.0
[p,z] => [r], 1.0
[t